In [ ]:
from sklearn.metrics import confusion_matrix

# Assuming you have the ground truth labels and predicted labels
y_true = [...]  # Ground truth labels
y_pred = [...]  # Predicted labels

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Extract TP, TN, FP, FN from the confusion matrix
TN = cm[0, 0]  # True negatives
FP = cm[0, 1]  # False positives
FN = cm[1, 0]  # False negatives
TP = cm[1, 1]  # True positives

# Print the results
print("True Negatives (TN):", TN)
print("False Positives (FP):", FP)
print("False Negatives (FN):", FN)
print("True Positives (TP):", TP)

The stats I decided to use:
Compute metrics
jaccard_score(y_true, y_pred, average=None)
confusion_matrix(y_true, y_pred)
f1_score(y_true, y_pred, average='weighted') #either weighted or micro
precision_score(y_true, y_pred, average='weighted')
recall_score(y_true, y_pred, average='weighted')
target_names = ['', 'class 1', 'class 2']
print(classification_report(y_true, y_pred, target_names=target_names))
hamming_loss(y_true, y_pred)
#precision_recall_fscore_support(y_true, y_pred, average='weighted')

In [ ]:
import torch.nn as nn
from math import *
import torch
num_channels = 1 #Depending if its multimodal or just fMRI 

class AlexNet3D_Dropout(nn.Module): #AlexNet3D_Deeper_Dropout
    def __init__(self, num_classes=2):  #This is dependent on three way = 3, two way = 2 and regression = 1
        super(AlexNet3D_Dropout, self).__init__() #ModuleList
        
        self.features = nn.Sequential(        
            nn.Conv3d(num_channels, 64*num_channels, kernel_size=5,
                      stride=1, padding=0, groups=num_channels), #kernel 5 and padding 0 for uni, kernel 3 and padding 1 for multi
            nn.BatchNorm3d(64*num_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=3),

            nn.Conv3d(64*num_channels, 128*num_channels, kernel_size=3,
                      stride=1, padding=0, groups=num_channels),
            nn.BatchNorm3d(128*num_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=3),

            nn.Conv3d(128*num_channels, 192*num_channels,
                      kernel_size=3, padding=1, groups=num_channels),
            nn.BatchNorm3d(192*num_channels),
            nn.ReLU(inplace=True),

            nn.Conv3d(192*num_channels, 384*num_channels,
                      kernel_size=3, padding=1, groups=num_channels),
            nn.BatchNorm3d(384*num_channels),
            nn.ReLU(inplace=True),

            nn.Conv3d(384*num_channels, 256*num_channels,
                      kernel_size=3, padding=1, groups=num_channels),
            nn.BatchNorm3d(256*num_channels),
            nn.ReLU(inplace=True),

            nn.Conv3d(256*num_channels, 256*num_channels,
                      kernel_size=3, padding=1, groups=num_channels),
            nn.BatchNorm3d(256*num_channels),
            nn.ReLU(inplace=True),
            #nn.MaxPool3d(kernel_size=3, stride=3)
            nn.AdaptiveAvgPool3d([1,1,1]),
        )
        self.classifier = nn.Sequential(nn.Dropout(),
                                        nn.Linear(256*num_channels, 64),
                                        nn.ReLU(inplace=True),
                                        nn.Dropout(),
                                        nn.Linear(64, num_classes),
                                        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

          
    def forward(self, x):
        xp = self.features(x)
        #for i, layer in enumerate(self.features):
            #x = layer(x)
            #print (i, x.size())
        x = xp.view(xp.size(0), -1)
        #print("final, ", x.shape)
        x = self.classifier(x)
        return [x, xp]


In [3]:
import torch
import torch.nn.functional as F

def addPadding(input_tensor):
    desired_size = (64, 64, 64)
    current_size = input_tensor.shape[-3:]
    padding = []
    for i in range(len(current_size)):
        diff = desired_size[i] - current_size[i]
        pad_before = diff // 2
        pad_after = diff - pad_before
        padding.extend([pad_before, pad_after])

    padding.extend([0] * (len(input_tensor.shape) - len(current_size) - 2))

    padded_size = input_tensor.shape[:-3] + tuple(desired_size)
    padded_image = F.pad(input_tensor, pad=padding[::-1], mode='constant', value=0)
    return padded_image


input_tensor = torch.randn(58, 1, 53, 63, 52)  # Input tensor
desired_size = (64, 64, 64)  # Desired size after padding

padded_image = addPadding(input_tensor, desired_size)

print(padded_image.shape)

torch.Size([58, 1, 64, 64, 64])
